In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

In [2]:
perturbed_input_fn = './perturbed_inputs.csv'
non_perturbed_input_fn = './non-perturbed_inputs.csv'

dirty_model = True
if dirty_model:
    pp_fn = './scores_perturbed:perturbed_final.csv'
    po_fn = './scores_non-perturbed:perturbed_final.csv'
else:
    pp_fn = './scores_perturbed:original_final.csv'
    po_fn = './scores_non-perturbed:original_final.csv'

In [3]:
p_on_p = pd.read_csv(pp_fn, header=None)
p_on_p.columns = ['example_idx', 'w1_prob', 'w2_prob', 'w1_rank', 'w2_rank']
p_on_p

,example_idx,w1_prob,w2_prob,w1_rank,w2_rank
0,920792,2.420170e-03,6.049702e-03,70,23
1,930635,2.659905e-02,4.024825e-02,3,1
2,716255,3.357364e-03,1.155391e-02,49,11
3,385756,3.253927e-03,5.506584e-03,41,13
4,976260,2.535612e-03,1.887857e-02,46,3
...,...,...,...,...,...
22495,587478,1.865367e-02,4.102668e-03,3,22
22496,651662,6.430567e-03,7.944838e-04,16,95
22497,720880,1.971224e-02,7.743267e-04,6,165
22498,526108,1.026309e-07,1.286599e-07,14990,13206


In [4]:
inputs = pd.read_csv(perturbed_input_fn)
pp_df = inputs.join(p_on_p)
pp_df['label'] = False      # for synonym
pp_df.head(1)

,example_index,text,sub_index,original,synonym,example_idx,w1_prob,w2_prob,w1_rank,w2_rank,label
0,920792,I also tried passing the parameters directly t...,276,nice,good,920792,0.00242,0.00605,70,23,False


In [5]:
p_on_o = pd.read_csv(po_fn, header=None)
p_on_o.columns = ['example_idx', 'w1_prob', 'w2_prob', 'w1_rank', 'w2_rank']
p_on_o

,example_idx,w1_prob,w2_prob,w1_rank,w2_rank
0,880143,0.006556,0.040112,14,2
1,897090,0.000082,0.001282,514,49
2,576122,0.009500,0.003830,15,41
3,20093,0.010976,0.003924,14,39
4,645693,0.000003,0.000325,4996,202
...,...,...,...,...,...
89995,539775,0.000166,0.000045,302,604
89996,239019,0.038890,0.000239,2,258
89997,60215,0.000063,0.000011,842,3528
89998,118039,0.000692,0.000535,56,68


In [6]:
inputs = pd.read_csv(non_perturbed_input_fn)
po_df = inputs.join(p_on_o)
po_df['label'] = True      # for original
po_df.head(1)

,example_index,text,sub_index,original,synonym,substituted,example_idx,w1_prob,w2_prob,w1_rank,w2_rank,label
0,880143,Purpose & Goals\n\nWhen Sol Worth and John Ada...,9701,nice,good,False,880143,0.006556,0.040112,14,2,True


In [7]:
df = pd.concat([pp_df, po_df])
df['label'].value_counts()

True     90000
False    22500
Name: label, dtype: int64

In [8]:
linear_adj = True

In [9]:
if linear_adj:
    logistic_models = {}
    for i, g in df.groupby("original"):
        temp_model = LinearRegression().fit(np.log(np.array(g["w1_prob"])).reshape(-1, 1), np.log(g["w2_prob"]))
        logistic_models[i] = temp_model

    def func(row):
        x = np.array(row["w1_prob"]).reshape(1, -1)
        prediction = np.exp(logistic_models[row["original"]].predict(np.log(x))[0])
        return prediction > row["w2_prob"]

    df["prediction"] = df.apply(func, axis=1)
else:
    df['prediction'] = df.w1_prob > df.w2_prob

In [10]:
df['correctness'] = df['label'] == df['prediction']
df.head(1)

,example_index,text,sub_index,original,synonym,example_idx,w1_prob,w2_prob,w1_rank,w2_rank,label,substituted,prediction,correctness
0,920792,I also tried passing the parameters directly t...,276,nice,good,920792,0.00242,0.00605,70,23,False,NaN,True,False


In [11]:
print('Accuracy on perturbed data only')
df[df.label == False][['original','correctness']].groupby(['original']).mean().sort_values('correctness', ascending=False)

Accuracy on perturbed data only


,correctness
original,
permit,0.636
totally,0.614
start,0.594
voice,0.582
help,0.580
nice,0.568
many,0.554
move,0.550
idea,0.550


In [12]:
accs = df[['original', 'label', 'correctness']].groupby(['original', 'label']).mean()

for w1 in df['original'].unique():
    acc = accs.loc[w1,True] * 0.5 + accs.loc[w1,False] * 0.5
    print(w1, acc)

nice correctness    0.53925
dtype: float64
size correctness    0.504
dtype: float64
way correctness    0.49775
dtype: float64
small correctness    0.53425
dtype: float64
guy correctness    0.4925
dtype: float64
start correctness    0.50425
dtype: float64
just correctness    0.51225
dtype: float64
first correctness    0.51675
dtype: float64
next correctness    0.5355
dtype: float64
permit correctness    0.525
dtype: float64
business correctness    0.50425
dtype: float64
monitor correctness    0.489
dtype: float64
big correctness    0.512
dtype: float64
more correctness    0.4905
dtype: float64
trouble correctness    0.501
dtype: float64
return correctness    0.4915
dtype: float64
lead correctness    0.5185
dtype: float64
very correctness    0.50025
dtype: float64
totally correctness    0.51125
dtype: float64
help correctness    0.50675
dtype: float64
area correctness    0.5195
dtype: float64
idea correctness    0.534
dtype: float64
device correctness    0.52325
dtype: float64
equal corr